https://github.com/tensorflow/serving/issues/310

In [1]:
import sys
sys.path.insert(0, '../aux/')
from beiras_aux import load_coded_dictionaries, predict_next_chars, clean_text
from keras.layers import Dense, Activation, GRU
from keras.models import Sequential
import os

Using TensorFlow backend.


In [2]:
# Input size of the network, the entry text must have the same length
window_size = 100
chars_to_indices, indices_to_chars = load_coded_dictionaries()

In [3]:
from keras.layers import InputLayer
def create_gru_model( num_chars):
    """
    Define the network
    :param
        numbers_chars .- Number chars using in the training process
    :return:
        model .- Model network defined
    """
    model = Sequential()
    # 1 Layer .- GRU layer 1 should be an GRU module with 200 hidden units
    model.add(GRU(200, input_shape=(window_size, num_chars), return_sequences=True))
    # 2 Layer .- GRU layer 2 should be an GRU module with 200 hidden units
    model.add(GRU(200))
    # 2 Layer .-  Dense, with number chars unit and softmax activation
    model.add(Dense(num_chars, activation='softmax'))
    return model

In [4]:
import keras.backend as K
K.set_learning_phase(0)

number_chars=len(chars_to_indices)
model=create_gru_model(number_chars)
model.load_weights('../model_weights/best_beiras_gru_textdata_weights.hdf5')


https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html

In [5]:
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter

In [6]:
export_path = "../export-tf"
import shutil
import os


if os.path.isdir(export_path):
    shutil.rmtree(export_path)
builder = saved_model_builder.SavedModelBuilder(export_path)

signature = predict_signature_def(inputs={'sequence': model.input},
                                  outputs={'scores': model.output})

with K.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tag_constants.SERVING],
                                         signature_def_map={'predict': signature})
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'../export-tf/saved_model.pb'


Prerequisitos installas tennsoflow servein
echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list

curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

sudo apt-get update && sudo apt-get install tensorflow-model-server


Dentro do directorio ..export-tf creemos dir 1 e movemos todo ahi, esta seria version 1.


tensorflow_model_server --port=9000 --model_name=mnist --model_base_path=/tmp/mnist_model/



In [7]:
print(model.input)

Tensor("gru_1_input:0", shape=(?, 100, 55), dtype=float32)


In [8]:
print(model.output)

Tensor("dense_1/Softmax:0", shape=(?, 55), dtype=float32)


https://gist.github.com/avloss/01e43d208fbdb2c5b4f9b50e71617cc8

In [1]:
import numpy as np
import pickle as pickle
import requests

URL = "http://localhost:9000/predict"

s = pickle.dumps({"x":x}, protocol=0)

DATA = {"model_name": "default",
            "input": requests.utils.quote(s)}

r = requests.get(URL, data=DATA)
return r.json()

ModuleNotFoundError: No module named 'requests'